# Demo of the baseline model

In [ ]:
import sys
sys.path.append("../")

import src.paths as PATHS
import src.constants as CONST

import src.data.data_handler as DH
import src.data.config as DATA_CONFIG
import src.utils as U

import src.model.baseline_model as BM

import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import MarkerCluster, TimestampedGeoJson

import altair as alt

import shapely
from shapely.geometry import Polygon, LineString, Point

from pprint import pprint

## 1. Gather all the inputs (normally user provided, or translated from user specifications)

### a. Configuration

Configuration drives the parameters of the feature creation. This would be a combination of user input (including defaults) and some internal parameters).

In the beaseline model case not all parameters are used, e.g. no need to define the remote data sources, as we only need the erosion data. Below we list the parameters relevant to the baseline model, even though we use their default values.

In [ ]:
# explicitly set the relevant parameters to the defaults, equivalent to running
# baseline_configuration = DATA_CONFIG.DataConfiguration()

baseline_configuration = DATA_CONFIG.DataConfiguration(
    no_of_points_for_distance_calculation=CONST.DEFAULT_NO_OF_POINTS_FOR_DISTANCE_CALCULATION,
    prediction_region_id_column_name=CONST.PREDICTION_REGION_ID,
    timestamp_column_name=CONST.TIMESTAMP,
    use_only_certain_river_bank_points=CONST.DEFAULT_USE_ONLY_CERTAIN_RIVER_BANK_POINTS,
)

In [ ]:
pprint(baseline_configuration)

### b. Prediction regions

Also called "scope" by Luke. A user defined dataset.

We have the data locally.

In [ ]:
luke_geospatial_data = PATHS.DATA_DIR / "luke_inputs_v3.gpkg"

scope_layer_name = "vlakken_scope"
prediction_regions = gpd.read_file(luke_geospatial_data, layer=scope_layer_name)

In [ ]:
print(prediction_regions.crs)
prediction_regions.info()

In [ ]:
prediction_regions.head()

### c. The local geospatial enrichment data

While we don't actually do any enrichment, we need the river centerline to be able to properly determine which ponts lie beyond the erosion border. Again, a user provided dataset.

We also have these locally.

In [ ]:
etienne_geospatial_data = PATHS.DATA_DIR / "Levering_erosie_data.gpkg"

centerline_layer_name = "Centreline_River"
centerline = gpd.read_file(etienne_geospatial_data, layer=centerline_layer_name)

# we need to align all the CRS
centerline.to_crs(prediction_regions.crs, inplace=True)

# the local geospatial enrichment data is a dictionary of geodataframes
# TODO: define this constant better, not via an operation
local_geospatial_data = {CONST.AggregationOperations.CENTERLINE_SHAPE.value: centerline}

In [ ]:
centerline.head()

### d. Erosion data

I.e. the points where the river bank was at different times. Used in model training.

We also have this locally.

In [ ]:
riverbank_layer_name = "punten_oever"
river_bank_locations = gpd.read_file(luke_geospatial_data, layer=riverbank_layer_name)

river_bank_locations.to_crs(prediction_regions.crs, inplace=True)

In [ ]:
river_bank_locations.sample(5)

### e. Erosion border

This is a line that has to have the right CRS. Can be user provided or internal.

We have it locally.

In [ ]:
fake_erosion_border = PATHS.DATA_DIR / "handdrawn_fake_erosion_border.geojson"

erosion_border_gdf = gpd.read_file(fake_erosion_border)
erosion_border_gdf.to_crs(prediction_regions.crs, inplace=True)

erosion_border = erosion_border_gdf.iloc[0]["geometry"]

In [ ]:
erosion_border

In [ ]:
mapa = folium.Map(location=[CONST.CENTRE_NL_LAT, CONST.CENTRE_NL_LON], zoom_start=CONST.DEFAULT_NL_ZOOM, control_scale=True)

# scope
fg_scope = folium.FeatureGroup(name="prediction regions (scope)", show=False).add_to(mapa)
for _, row in prediction_regions.to_crs(epsg=CONST.EPSG_WGS84).iterrows():
    polygon = folium.GeoJson(row["geometry"])
    folium.Popup(row[CONST.PREDICTION_REGION_ID]).add_to(polygon)
    polygon.add_to(fg_scope)

# river centerline
fg_centerline = folium.FeatureGroup(name="river centerline", show=False).add_to(mapa)
folium.GeoJson(centerline["geometry"].to_crs(epsg=CONST.EPSG_WGS84)).add_to(fg_centerline)

# river bank locations
fg_bank = folium.FeatureGroup(name="river bank", show=False).add_to(mapa)
# folium.GeoJson(river_bank_locations["geometry"].to_crs(epsg=CONST.EPSG_WGS84)).add_to(fg_bank)

# Add points to the map
for idx, row in river_bank_locations.to_crs(epsg=CONST.EPSG_WGS84).iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius = 2,
        color="blue" if row[CONST.TIMESTAMP] == 3 else "orange",
        opacity=0.5,
    ).add_to(fg_bank)


# erosion border
fg_border = folium.FeatureGroup(name="erosion border", show=False).add_to(mapa)
folium.GeoJson(erosion_border_gdf["geometry"].to_crs(epsg=CONST.EPSG_WGS84)).add_to(fg_border)

folium.LayerControl().add_to(mapa)

mapa

## 2. Create a prediction model

The baseline model averages the changes in the river bank position in time for each region.

In [ ]:
data_handler = DH.DataHandler(
    config=baseline_configuration,
    prediction_regions=prediction_regions,
    local_data_for_enrichment=local_geospatial_data,
    erosion_data=river_bank_locations,
    erosion_border=erosion_border,
)

data_handler.process_erosion_features()

In [ ]:
print(data_handler.processed_erosion_data.shape)
data_handler.processed_erosion_data.head()

In [ ]:
baseline_model = BM.BaselineErosionModel(
    config=baseline_configuration,
    training_data=data_handler.processed_erosion_data,
    verbose=True,
)

baseline_model.train()

In [ ]:
baseline_model.model

## 3. Run and visualize the prediction

In [ ]:
data_for_prediction = data_handler.processed_erosion_data.copy()

latest_timestamp = data_for_prediction.index.get_level_values(CONST.TIMESTAMP).unique().max()

data_for_prediction = data_for_prediction[data_for_prediction.index.get_level_values(CONST.TIMESTAMP) == latest_timestamp]

In [ ]:
prediction = baseline_model.predict(data_for_prediction, prediction_length=10)

prediction

In [ ]:
def project_scope_onto_erosion_border(scope_polygon: Polygon, erosion_border: LineString) -> LineString:
    """Get a section of the erosion border that corresponds to the scope polygon onto it.
    
    :param scope_polygon: polygon within which we predict the bank erosion
    :param erosion_border: the erosion border
    :returns: a section of the erosion border

    NOTE: we assume that the erosion border is generally larger than the scope polygon

    TODO: right now this only creates a straight line from one edge point to the other. Instead, do we want this to copy the erosion line or do sth else?
    """
    scope_points = list(scope_polygon.exterior.coords)

    projected_distances = []
    for scope_point in scope_points:
        distance_along_erosion_border = erosion_border.project(Point(scope_point))
        projected_distances.append(distance_along_erosion_border)

    min_distance = min(projected_distances)
    max_distance = max(projected_distances)

    starting_point = erosion_border.interpolate(min_distance)
    ending_point = erosion_border.interpolate(max_distance)

    projection_line = LineString([starting_point, ending_point])

    return projection_line


def move_projected_scope_to_side(projected_scope: LineString, distance: float, river_centerline: LineString) -> LineString:
    """Using the distance from the erosion border, get where the projected scope polygon would lie as a surrogate to the river bank.
    
    :param projected_scope: line that projects the scope polygon onto the erosion border
    :param distance: distance between the (predicted) river bank and the erosion border. Positive values are "good", the bank being on the correct side, for negative values the river bank went over
    :param river_centerline: the centerline to determine the direction of shifting

    NOTE: we assume that the projected_scope is a line almost exactly copying the existing erosion_border. Hence, we shift it a bit - say, one metre, assuming it will never cross the river centerline like this,
          and if it gets closer to centerline we know this is the direction we should be shifting if the distance is positive.

    TODO: can we assume the direction of lines in the data to help with the moving left or right? E.g. if both the river centerline and the erosion border go from upstream down, that might help.
    """
    small_shift = 1  # metre
    slightly_shifted_projected_scope = projected_scope.offset_curve(small_shift)

    # a small shift by a positive value should move the line closer to the river centerline, otherwise we need to swap the distance
    distance_multiplier = 1 if river_centerline.distance(slightly_shifted_projected_scope) < river_centerline.distance(projected_scope) else -1

    return projected_scope.offset_curve(distance_multiplier * distance)

        

In [ ]:
def process_prediction(prediction_dataframe, measurement_year: int = 2024) -> gpd.GeoDataFrame:
    """Turn the prediction into a geodataframe.
    
    TODO: don't assume that we only have a single ahn version
    """
    TIME_COLUMN = "time_column"
    DISTANCE = "distance_to_erosion"
    YEAR = "year"
    TIMESTAMP = "timestamp"
    
    processed_prediction = pd.melt(prediction_dataframe, var_name=TIME_COLUMN, value_name=DISTANCE, ignore_index=False).reset_index().copy()
    processed_prediction[YEAR] = processed_prediction[TIME_COLUMN].map(lambda x: int(x.split('_')[-1]) + measurement_year)
    processed_prediction[TIMESTAMP] = processed_prediction[YEAR].map(lambda x: f"{x}-1-1")
    processed_prediction[TIMESTAMP] = pd.to_datetime(processed_prediction[TIMESTAMP])
    processed_prediction.drop([CONST.TIMESTAMP, TIME_COLUMN, YEAR], axis=1, inplace=True)

    return processed_prediction

    

In [ ]:
processed_prediction = process_prediction(prediction)
processed_prediction = processed_prediction.merge(prediction_regions, on=CONST.PREDICTION_REGION_ID)
processed_prediction.rename(columns={"geometry": "prediction_region"}, inplace=True)

processed_prediction["relevant_centerline"] = processed_prediction["prediction_region"].map(lambda x: U.get_relevant_centerline(x, centerline))
processed_prediction["projected_scope"] = processed_prediction["prediction_region"].map(lambda x: project_scope_onto_erosion_border(x, erosion_border))

processed_prediction["predicted_riverbank"] = processed_prediction.apply(lambda x: move_projected_scope_to_side(x["projected_scope"], x["distance_to_erosion"], x["relevant_centerline"]), axis=1)

processed_prediction.drop(["prediction_region", "relevant_centerline", "projected_scope"], axis=1, inplace=True)
processed_prediction = gpd.GeoDataFrame(processed_prediction, crs=prediction_regions.crs, geometry="predicted_riverbank")
                                                                     
processed_prediction.head()

In [ ]:
mapa = folium.Map(location=[CONST.CENTRE_NL_LAT, CONST.CENTRE_NL_LON], zoom_start=CONST.DEFAULT_NL_ZOOM, control_scale=True)

# predicted_riverbank
fg_bank = folium.FeatureGroup(name="predicted river bank", show=False).add_to(mapa)
geojson_features = []
for _, row in processed_prediction.to_crs(epsg=CONST.EPSG_WGS84).iterrows():
    # TODO: lines are not rendered even though this looks exactly like the example in the docs where lines are rendered?!
    line = row["predicted_riverbank"].__geo_interface__
    line["coordinates"] = [list(a) for a in line['coordinates']]
    geojson_features.append({
        "type": "Feature",
        "geometry": line,
        "properties": {
            "times": [row["timestamp"].strftime("%Y-%m-%dT%H:%M:%S")],
            "style": {"color": "green" if row["distance_to_erosion"] > 0 else "red", "weight": 3, "opacity":0.6},
        },
    })

geojson_data = {"type": "FeatureCollection", "features": geojson_features}
TimestampedGeoJson(
    geojson_data,
    period="P1Y",
    duration="P6M",
    transition_time=200,  # Milliseconds between frames
    loop=False,            # Loop animation
    auto_play=False,      # Start playing automatically
    loop_button=True,
).add_to(mapa)

# erosion border
fg_border = folium.FeatureGroup(name="erosion border", show=False).add_to(mapa)
folium.GeoJson(erosion_border_gdf["geometry"].to_crs(epsg=CONST.EPSG_WGS84)).add_to(fg_border)

# the prediction regions with the predicted river banks
fg_scope = folium.FeatureGroup(name="prediction regions", show=True).add_to(mapa)

measurement_year = 2024
predicted_years = [measurement_year + int(col.split("_")[-1]) for col in prediction.columns]

for ind,row in prediction_regions.to_crs(CONST.EPSG_WGS84).iterrows():
    predicted_values = prediction[prediction.index.get_level_values(CONST.PREDICTION_REGION_ID) == row[CONST.PREDICTION_REGION_ID]].iloc[0].values
    data = pd.DataFrame({"year": predicted_years, "distance to erosion border (m)": predicted_values})
    color = "orange" if (data["distance to erosion border (m)"] < 0).any() else "blue"

    chart = alt.Chart(data, title=row[CONST.PREDICTION_REGION_ID]).mark_line(point=True).encode(x="year", y="distance to erosion border (m)")
    horizontal_line = alt.Chart(pd.DataFrame({"y": [0]})).mark_rule(strokeDash=[5, 5], color="black").encode(y="y")
    chart = chart + horizontal_line
    
    vega_chart = folium.VegaLite(chart, width="100%", height="100%",)
    popup = folium.Popup()
    vega_chart.add_to(popup)

    shape = shapely.to_geojson(row["geometry"])
    
    polygon = folium.GeoJson(
        row["geometry"],
        style_function=lambda feature, color=color:  {"color": color, "fillcolor": color}
    )
    popup.add_to(polygon)

    polygon.add_to(fg_scope)
    

folium.LayerControl().add_to(mapa)

mapa